In [1]:

import numpy as np
import tensorflow as tf
import pickle

from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-06-01 20:14:16.180947: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-01 20:14:16.351087: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-01 20:14:16.444014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748808856.559915    2749 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748808856.591921    2749 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748808856.847264    2749 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
!pip install --upgrade numpy

  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [3]:
import numpy
print(numpy.__version__)
print(numpy.__file__)


1.26.4
/workspaces/General-ChatBot/.conda/lib/python3.12/site-packages/numpy/__init__.py


In [5]:
from flask import Flask, request, jsonify
from sklearn.preprocessing import LabelEncoder

In [6]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Load the model, tokenizer (words.pkl), and classes (classes.pkl)
model = tf.keras.models.load_model('/workspaces/General-ChatBot/chatbot/chatbot_model.h5')

# Load tokenizer and label encoder
with open("/workspaces/General-ChatBot/chatbot/words.pkl", 'rb') as f:
    words = pickle.load(f)

with open('/workspaces/General-ChatBot/chatbot/classes.pkl', 'rb') as f:
    classes = pickle.load(f)


/workspaces/General-ChatBot/.conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-01 20:25:05.615767: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/workspaces/General-ChatBot/.conda/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [9]:
 #Initialize Flask app
app = Flask(__name__)

# Preprocess the user input
def preprocess_input(user_input):
    # Tokenize and encode the user input
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(words)
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, maxlen=20, padding='post')  # Adjust padding if needed
    return padded_sequence

# Get the chatbot response
def chatbot_response(user_input):
    processed_input = preprocess_input(user_input)
    prediction = model.predict(processed_input)
    predicted_class = np.argmax(prediction)
    
    # Map prediction back to a class
    response_class = classes[predicted_class]  
    return response_class

# Flask route for chatbot interaction
@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get("message")
    if not user_input:
        return jsonify({"error": "No input provided"}), 400
    
    response = chatbot_response(user_input)
    return jsonify({"response": response})
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, this is the homepage of your chatbot!"

# Your existing chatbot code here...

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
  # Bind to all network interfaces

   


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 20:25:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 20:25:17] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pickle
import tensorflow as tf
import json
import re
from sympy import symbols, diff, integrate, log, exp, sympify
import sympy as sp

# Define symbols for calculus operations
x = symbols('x')

# Function to handle basic arithmetic calculations
def calculate_math(expression):
    try:
        match = re.match(r"(\d+)\s*([+\-*/])\s*(\d+)", expression)
        if match:
            num1 = int(match.group(1))
            operator = match.group(2)
            num2 = int(match.group(3))

            if operator == '+':
                return num1 + num2
            elif operator == '-':
                return num1 - num2
            elif operator == '*':
                return num1 * num2
            elif operator == '/':
                return num1 / num2
        else:
            return "Sorry, I couldn't understand the calculation."
    except Exception as e:
        return "There was an error with the calculation."

# Function for advanced math operations (differentiation, integration, log, exponentials)
def calculate_expression(user_message):
    try:
        # Differentiation
        diff_match = re.search(r'differentiate\s(.+)', user_message, re.IGNORECASE)
        if diff_match:
            expression = sympify(diff_match.group(1))
            result = diff(expression, x)
            return f"The derivative of {expression} with respect to x is {result}"

        # Integration
        int_match = re.search(r'integrate\s(.+)', user_message, re.IGNORECASE)
        if int_match:
            expression = sympify(int_match.group(1))
            result = integrate(expression, x)
            return f"The integral of {expression} with respect to x is {result}"

        # Logarithm
        log_match = re.search(r'log\s*\(\s*(\d+)\s*\)', user_message, re.IGNORECASE)
        if log_match:
            number = int(log_match.group(1))
            result = log(number)
            return f"The logarithm of {number} is {result}"

        # Exponential
        exp_match = re.search(r'e\^\s*(\d+)', user_message, re.IGNORECASE)
        if exp_match:
            power = int(exp_match.group(1))
            result = sp.exp(power).evalf()  # Evaluate the exponential as a float
            return f"The result of e^{power} is {result}"

        return "Sorry, I couldn't understand the calculation."
    except Exception as e:
        return "There was an error with the calculation."

# Load the chatbot model and necessary files
model = tf.keras.models.load_model(r'/workspaces/General-ChatBot/chatbot/chatbot_model.h5')
words = pickle.load(open(r'/workspaces/General-ChatBot/chatbot/words.pkl', 'rb'))
classes = pickle.load(open(r'/workspaces/General-ChatBot/chatbot/classes.pkl', 'rb'))

# Load intents.json with utf-8 encoding
with open(r'/workspaces/General-ChatBot/chatbot/intents.json', encoding='utf-8') as file:
    intents = json.load(file)


app = Flask(__name__)

# Homepage route
@app.route('/')
def home():
    return render_template('home.html')

# Chatbot page route
@app.route('/chatbot')
def chatbot():
    return render_template('chat.html')

# Route to get chatbot responses
@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json['message']
    
    # Check if the user message is an advanced math question
    advanced_math_response = calculate_expression(user_message)
    if "Sorry" not in advanced_math_response:
        return jsonify({"response": advanced_math_response})
    
    # Check if the user message is a basic math question
    if re.search(r'\d+\s*[+\-*/]\s*\d+', user_message):
        result = calculate_math(user_message)
        return jsonify({"response": f"The answer is {result}"})
    
    # Process regular chatbot responses
    response = generate_chatbot_response(user_message)
    return jsonify({'response': response})

# Function to process user input and return chatbot response
def generate_chatbot_response(message):
    message_tokens = tokenize_sentence(message)
    prediction = model.predict(np.array([message_tokens]))[0]
    predicted_class = np.argmax(prediction)
    response = get_response_for_class(predicted_class)
    return response

# Tokenize the sentence and convert it into a bag of words
def tokenize_sentence(sentence):
    sentence_words = sentence.lower().split()
    bag_of_words = [0] * len(words)
    for word in sentence_words:
        if word in words:
            index = words.index(word)
            bag_of_words[index] = 1
    return np.array(bag_of_words)

# Function to get the response based on predicted class
def get_response_for_class(predicted_class):
    intent_label = classes[predicted_class]
    for intent in intents['intents']:
        if intent['tag'] == intent_label:
            return np.random.choice(intent['responses'])
    return "Sorry, I don't understand that."

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


2025-06-27 08:15:55.780954: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-27 08:16:00.972551: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-27 08:16:05.132092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751012168.595095    5521 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751012169.574813    5521 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751012176.581821    5521 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

KeyboardInterrupt: 